<a href="https://colab.research.google.com/github/Decko048/DKD_048/blob/main/Trial_People_Analytics_1-251220.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load Modul & Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

In [ ]:
# Load the trining data
tr_df=pd.read_csv('https://decko048.github.io/dataset/train.csv')
ts_df=pd.read_csv('https://decko048.github.io/dataset/test.csv')

print(tr_df.shape)
print(ts_df.shape)
pd.set_option('display.max_columns',30)
tr_df.head(8)

In [ ]:
tr_df.info()

In [ ]:
tr_df.describe().job_duration_in_current_job_level

In [ ]:
plt.hist(tr_df.job_duration_in_current_job_level, color='green')
plt.xlabel('Lama Kerja di Jabatan Terakhir')
plt.ylabel('Tahun')

In [ ]:
tr_df.corr().job_duration_in_current_job_level.hist()

In [ ]:
tr_df.corr().loc[(tr_df.corr().job_duration_in_current_job_level<-0.3)
 | (tr_df.corr().job_duration_in_current_job_level>0.3)]['job_duration_in_current_job_level']

In [ ]:
sns.lmplot(x='job_duration_in_current_job_level', y='Best Performance', data=tr_df)

In [ ]:
tr_df.describe().age

In [ ]:
plt.hist(tr_df.age, color='green')
plt.xlabel('Usia')
plt.ylabel('Tahun')

In [ ]:
tr_df.corr().age.hist()

In [ ]:
tr_df.corr().loc[(tr_df.corr().age<-0.3)
 | (tr_df.corr().age>0.3)]['age']

In [ ]:
sns.lmplot(x='age', y='Best Performance', data=tr_df)

In [ ]:
# Missing Values in each columns

sns.set(style="darkgrid")
pd.DataFrame(tr_df.isna().sum().sort_values(ascending=True), columns=['Nilai Hilang']
             ).plot(kind='barh', figsize=(10,6))
plt.show()

In [ ]:
# Unique values in each columns

sns.set(style="darkgrid")
pd.DataFrame(tr_df.drop(columns=['Best Performance']).nunique().sort_values(ascending=True),
             columns=['Nilai Unique']).plot(kind='barh',figsize=(10,6), logx=True)
plt.show()


# **Exploratory Data Analysis of Training Data**

Number of job_level in different categories accross both the type of target "Best Performance"

In [ ]:
list_df =['job_duration_in_current_job_level','job_duration_in_current_person_level',
          'job_duration_in_current_branch','sick_leaves','age','job_duration_from_training',
          'assign_of_otherposition','annual leave','branch_rotation','job_rotation',
          'person_level','number_of_dependences (female)','number_of_dependences (male)',
          'job_level','Employee_type','Employee_status','gender','marital_status_maried(Y/N)']

sns.set(style="darkgrid")
fig, ax = plt.subplots(9,2, figsize=(18,20))
for i, ax in zip(range(18), ax.flat):
    col = list_df[i]
    sns.countplot(x=col, hue='Best Performance', data=tr_df, ax=ax)
plt.show()

In [ ]:
# Correlation among various numerical Features

plt.figure(figsize=(8,8))
sns.heatmap(tr_df.drop(columns=['Best Performance']).corr(), annot=True,
            square=True)
plt.show()

In [ ]:
# Number of Employees with different data=tr_df

sns.set(style="darkgrid")
plt.figure(figsize=(10,6))
sns.countplot(x='job_duration_in_current_job_level', hue='Best Performance', data=tr_df)
plt.show()

In [ ]:
sns.lineplot(y='Best Performance', x='job_duration_in_current_job_level', data=tr_df)

In [ ]:
sns.lineplot(y='Best Performance', x='age', data=tr_df)

In [ ]:
# Number of Employee with different "JOB DURATION IN CURRENT PERSON LEVEL"

sns.set(style="darkgrid")
plt.figure(figsize=(10,6))
sns.countplot(x='job_duration_in_current_person_level', hue='Best Performance', data=tr_df)
plt.show()

In [ ]:
# Number of Employee with different "JOB DURATION IN CURRENT BRANCH"

sns.set(style="darkgrid")
plt.figure(figsize=(10,6))
sns.countplot(x='job_duration_in_current_branch', hue='Best Performance', data=tr_df)
plt.show()

In [ ]:
# Number of Employee with different "AGE"

sns.set(style="darkgrid")
plt.figure(figsize=(10,6))
age_df=sns.countplot(x='age', hue='Best Performance', data=tr_df)
plt.show()

In [ ]:
# Number of Employee with different "SICK LEAVES"

sns.set(style="darkgrid")
plt.figure(figsize=(10,6))
sns.countplot(x='sick_leaves', hue='Best Performance', data=tr_df)
plt.show()

In [ ]:
# Number of Employee with different "JOB DURATION FROM TRAINING"

sns.set(style="darkgrid")
plt.figure(figsize=(10,6))
sns.countplot(x='job_duration_from_training', hue='Best Performance', data=tr_df)
plt.show()

# **DEEP LEARNING Variables Transformamtion** _20-12-25_V1.0

Untuk tahapan pengelompokan dengan menormalkan data numerik pada skala 0 dan 1 saja menggunakan fungsi Min-Max.
Nilai atribut yg berkelanjutan dikelompokkan ke dalam beberapa kelompok untuk membatasi penyebaran data.
Untuk mempercepat pembelajaran algoritma, dilakukan perubahan nilai kategori "Best Performance" menjadi nilai integer berurutan

In [ ]:
# Deep Learning for People Analysis

import numpy
numpy.random.seed(7)
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Adam

fn_df=tr_df


In [ ]:
bpdf=fn_df['age'].unique()
bpdf_mapping=dict(zip(bpdf, range(0, len(bpdf) + 1)))
fn_df['age_int']=fn_df['age'].map(bpdf_mapping).astype(int)
depart=fn_df['age'].unique()
depart_mapping=dict(zip(depart, range(0,len(depart)+1)))
fn_df['depart_int']=fn_df['age'].map(depart_mapping).astype(int)
fn_df=fn_df.drop(["age","age"], axis=1)

train, test = train_test_split(fn_df, test_size=0.2)

X_train=train.drop(["Best Performance"], axis=1)
Y_train=train["Best Performance"]
X_train=test.drop(["Best Performance"], axis=1)
Y_train=test["Best Performance"]

print(bpdf_mapping)
print(depart_mapping)

print(train.shape)

In [ ]:

def create_model(neurons=16, dropout_rate=0.2, weight_constraint=3,activation='relu',learn_rate=0.01, momentum=0.4):
    
    #create model
    model_dl = Sequential()
    model_dl.add(Dense(neurons, input_dim=9, kernel_initializer='uniform',
                        activation=activation, kernel_constraint=maxnorm(weight_constraint)))
    model_dl.add(Dense(neurons, kernel_initializer='uniform', 
                     activation=activation, kernel_constraint=maxnorm(weight_constraint)))
    model_dl.add(Dense(neurons, kernel_initializer='uniform', 
                     activation=activation, kernel_constraint=maxnorm(weight_constraint)))
    model_dl.add(Dense(neurons, kernel_initializer='uniform', 
                     activation=activation, kernel_constraint=maxnorm(weight_constraint)))
    model_dl.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    # Compile model
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model_dl.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model_dl

best_model = create_model()

history_best = best_model.fit(X_train, Y_train, epochs= 20, batch_size= 20, validation_split=0.1, verbose= 1)


In [ ]:
# list all data in history

print(history_best.history.keys())
# summarize history for accuracy

plt.plot(history_best.history['acc'])
plt.plot(history_best.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history_best.history['loss'])
plt.plot(history_best.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
def create_model_easy():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=9, activation='sigmoid'))
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dropout(0,7))
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dropout(0,7))
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dropout(0,7))
    model.add(Dense(16, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = Adam(decay=0.001)
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Fit the model
my_model = create_model_easy()
history = my_model.fit(X_train, Y_train, epochs=50, batch_size=15, validation_split=0.2, verbose=1)

print("Model finished")

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
scores = my_model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (my_model.metrics_names[0], scores[0]*100))
print("\n%s: %.2f%%" % (my_model.metrics_names[1], scores[1]*100))